In [8]:
%load_ext nvcc4jupyter

from nvcc4jupyter import set_defaults
set_defaults(compiler_args='-arch=sm_100a -Xptxas=-v -O0')

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


# GOD I AM SO HAPPY, SO HAPPY, THAT JUPYTER IN REMOTE SSH IS WORKING AGAIN! LETS DO SOME SMEM STUFF BABYYYYYY. 

In [10]:
%%cuda

#include<stdio.h> 
#include<stdlib.h>
#include<cuda.h>
#include<cuda_runtime.h> 

constexpr int N_iter = 100000;
constexpr int N_warps = 32;
constexpr int N_lanes = 32;
constexpr int N_blocks = 1;

__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

__global__ void smem_no_bc()
{
  __shared__ float A[N_warps][N_lanes];
  int warp_id = threadIdx.y; 
  int lane_id = threadIdx.x; 
  
  float* ptr = &A[warp_id][lane_id];
  int addr = (int)ptr & 0xFFFF; //#0x is just a prefix meaning its hexadecimal 
  //# each F is a 15, and each digit is worth that power of 16. 
  //# in fact this number is 2^16 + 1, which means we are bit masking with the and 
  //# and extracting the first 16 digits of our pointer. but why? 
  /*
  """
  well really, each address is worth a byte (8 bits) and so how many bytes do we have in this shared memory?
  we have 8*32 bytes, so we have 8*8*32 bits, so its enough for our adress to be that big 
  well 8*8*32 = 2^6 * 2^5 = 2^11, 
  
  but the adress itself, if you do addr + 1, you are moving 8 bits (or 1 byte) 
  so then, if we have 2^16 - 1 addrs, we are guuci, but idk whats more to this nvm
  """
  */
  float r1;
  int n_clocks = 0;
  unsigned long long start;
  unsigned long long end;
  
  
  for (int j = 0; j < N_iter; j++) {
      
      start = get_clock64();
      
      asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                      : "=f"(r1)
                      : "r"(addr));
      
      end = get_clock64();
      
      if (threadIdx.x == 0 && threadIdx.y == 0)
        {
          n_clocks += end - start;
        }
  }
  
  __syncthreads(); 
  if (threadIdx.x == 0 && threadIdx.y == 0)
  {
    printf("%d \n", n_clocks);
  }
  
}

int main()
{
   
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

 
  cudaEventRecord(start, 0);

 
  //#weirdly enough, it is actually threadidx.x that has stride one, even tho its 
  //#on the left of the tuple. so we gotta be careful about that launch
  dim3 threads(N_lanes, N_warps);
  dim3 grid(N_blocks);
  
  smem_no_bc<<<grid, threads>>>();


  cudaEventRecord(stop, 0);
  cudaEventSynchronize(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("GPU Kernel time: %f ms\n", milliseconds);
  
  
  /*
  """
  so how do we calculate our memory transfer speed, well, we have N_warps warps, 
  so per iteration, we issue N_warp loads per bank, so our speed will be 
  N_loads/bank/second. 
  so for N_iter iterations, 
  smem_bank_speed = (N_warps*N_iter)/time 
  """
  */
  float seconds = milliseconds*0.001;
  float mega = 1000000.0;
  printf("mega_loads/bank/second = %f \n", (N_warps*N_iter)/(seconds*mega));
  
  int deviceId;
  cudaGetDevice(&deviceId);
  
  cudaDeviceProp prop;
  cudaGetDeviceProperties(&prop, deviceId);
  

  double smClockMHz = prop.clockRate / 1000.0;
  
  printf("Device: %s\n", prop.name);
  printf("Base SM Clock Rate: %.2f MHz\n", smClockMHz);
  printf("latency? %f", (2711953.0)/(N_iter));
}

2711953 
GPU Kernel time: 25.872383 ms
mega_loads/bank/second = 123.684006 
Device: NVIDIA GeForce RTX 5090
Base SM Clock Rate: 2437.00 MHz
latency? 27.119530


In [5]:
"""
one wonders why I am not getting 1 load per bank per clock cycle. 

"""

'\none wonders why I am not getting 1 load per bank per clock cycle. \n\n'

In [20]:
%%cuda
#include <stdio.h> 
#include <stdlib.h>
#include <cuda_runtime.h> 
#include <device_launch_parameters.h>

// Define a safe macro for CUDA error checking
#define CUDA_CHECK(call)                                                          \
    do {                                                                          \
        cudaError_t err = call;                                                   \
        if (err != cudaSuccess) {                                                 \
            fprintf(stderr, "CUDA error at %s:%d: %s\n",                          \
                    __FILE__, __LINE__, cudaGetErrorString(err));                 \
            exit(EXIT_FAILURE);                                                   \
        }                                                                         \
    } while (0)

constexpr int N_iter = 2000;
constexpr int N_threads = 1;
constexpr int N_blocks = 1;

// 64-bit clock counter: This is correct for PTX `clock64`.
__device__ __forceinline__ unsigned long long get_clock64() {
    unsigned long long clock_val;
    asm volatile("mov.u64 %0, %%clock64;" : "=l"(clock_val));
    return clock_val;
}

// NOTE: Kernel signature requires 4 arguments.
__global__ void scalar_smem_latency (float* In, float* Out, unsigned long long *starts, unsigned long long *stops)
{
  int t = threadIdx.x; 
  __shared__ float S[N_iter]; 
  float r1; 
 
  unsigned long long s; 
  unsigned long long e; 
  
  for (int j = 0; j < N_iter; j++)

  { 
      if(j < 10)
      {
        printf("%f  ", In[t+j]);
      }

      S[t+j] = In[t+j];
  }
  
  __syncthreads();
  
  for (int j = 0; j < N_iter; j+=8)
  {
    float* ptr_0 = &S[t + j]; 
    int addr_0 = (int)ptr_0 & 0xFFFF;
    float* ptr_1 = &S[t + j + 1]; 
    int addr_1 = (int)ptr_1 & 0xFFFF;
    float* ptr_2 = &S[t + j + 2]; 
    int addr_2 = (int)ptr_2 & 0xFFFF;
    float* ptr_3 = &S[t + j + 3]; 
    int addr_3 = (int)ptr_3 & 0xFFFF;
    float* ptr_4 = &S[t + j + 4]; 
    int addr_4 = (int)ptr_4 & 0xFFFF;
    float* ptr_5 = &S[t + j + 5]; 
    int addr_5 = (int)ptr_5 & 0xFFFF;
    float* ptr_6 = &S[t + j + 6]; 
    int addr_6 = (int)ptr_6 & 0xFFFF;
    float* ptr_7 = &S[t + j + 7]; 
    int addr_7 = (int)ptr_7 & 0xFFFF;
    Out[t + j] = r1;
    s = get_clock64();
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_0));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_1));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                      : "=f"(r1)
                      : "r"(addr_2));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_3));
        asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_4));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_6));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                      : "=f"(r1)
                      : "r"(addr_6));
    asm volatile ("ld.volatile.shared.f32 %0, [%1]; \n"
                    : "=f"(r1)
                    : "r"(addr_7));
      
    e = get_clock64();
    starts[t + j] = s; 
    stops[t + j] = e;
    Out[t + j + 7] = r1;

  }
  
}


int main()
{
    //# --- 1. Variable Declarations and Size Calculations ---
    unsigned long long *h_start, *h_stop;
    unsigned long long *d_start, *d_stop;
    
    //# Size for cycle arrays (N_iter measurements for 1 thread)
    size_t size_cycles = N_iter * sizeof(unsigned long long);
    
    //# Size for I/O float arrays
    size_t size_IO = N_iter * sizeof(float);
    float* h_In, *h_out;
    float* D_In, *D_out; // Device pointers for In and Out (required by kernel signature)

    //# --- 2. Host and Device Memory Allocation and Initialization ---
    
    //# Allocate Pinned Host memory for In and Out (for setup)
    CUDA_CHECK(cudaHostAlloc(&h_In, size_IO, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_out, size_IO, cudaHostAllocDefault)); 
    
    //# Corrected loop syntax: semicolon instead of comma in the condition
    for (int i = 0; i < N_iter; i++) 
    {
        h_In[i] = (float) (i+1) / (100.0); 
        h_out[i] = 0.0f; // #Use 0.0f for float literal
    }
    
    //# Allocate Pinned Host memory for start/stop cycles
    CUDA_CHECK(cudaHostAlloc(&h_start, size_cycles, cudaHostAllocDefault));
    CUDA_CHECK(cudaHostAlloc(&h_stop, size_cycles, cudaHostAllocDefault));
    
    // Allocate Device memory for all 4 kernel arguments
    CUDA_CHECK(cudaMalloc(&d_start, size_cycles));
    CUDA_CHECK(cudaMalloc(&d_stop, size_cycles));
    CUDA_CHECK(cudaMalloc(&D_In, size_IO));
    CUDA_CHECK(cudaMalloc(&D_out, size_IO));

    // Copy input data to the device (D_In)
    CUDA_CHECK(cudaMemcpy(D_In, h_In, size_IO, cudaMemcpyHostToDevice));
    
    // --- 3. Kernel Launch ---

    printf("Launching kernel with N_blocks=%d and N_threads=%d...\n", N_blocks, N_threads);

    // Launch the kernel with all 4 required arguments
    scalar_smem_latency<<<N_blocks, N_threads>>>(D_In, D_out, d_start, d_stop);
    
    // Wait for the kernel to finish
    CUDA_CHECK(cudaDeviceSynchronize());
    
    // --- 4. Data Transfer and Processing ---
    
    // Copy the cycle counters back to the host
    CUDA_CHECK(cudaMemcpy(h_start, d_start, size_cycles, cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_stop, d_stop, size_cycles, cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_out, D_out, size_IO, cudaMemcpyDeviceToHost));

    float avg = 0.0;
    
    for (int i = 0; i < N_iter; i+=8)
    {
        // Safe subtraction using unsigned long long
        unsigned long long diff = h_stop[i] - h_start[i];
        printf("value of r1 at iteration %d: %f \n", i, h_out[i]);
        printf("Latency at iteration %d: %llu cycles \n", i, diff);
        avg += ((float)diff/8.0);

    }
    printf("average latency %f \n", avg/250.0);
    // --- 5. Cleanup ---
    
    cudaFree(d_start);
    cudaFree(d_stop);
    cudaFree(D_In);
    cudaFree(D_out);
    
    cudaFreeHost(h_start);
    cudaFreeHost(h_stop);
    cudaFreeHost(h_In);
    cudaFreeHost(h_out);
    
    return 0;
}

Launching kernel with N_blocks=1 and N_threads=1...
0.010000  0.020000  0.030000  0.040000  0.050000  0.060000  0.070000  0.080000  0.090000  0.100000  value of r1 at iteration 0: 0.000000 
Latency at iteration 0: 45 cycles 
value of r1 at iteration 8: 0.080000 
Latency at iteration 8: 53 cycles 
value of r1 at iteration 16: 0.160000 
Latency at iteration 16: 45 cycles 
value of r1 at iteration 24: 0.240000 
Latency at iteration 24: 53 cycles 
value of r1 at iteration 32: 0.320000 
Latency at iteration 32: 45 cycles 
value of r1 at iteration 40: 0.400000 
Latency at iteration 40: 53 cycles 
value of r1 at iteration 48: 0.480000 
Latency at iteration 48: 45 cycles 
value of r1 at iteration 56: 0.560000 
Latency at iteration 56: 53 cycles 
value of r1 at iteration 64: 0.640000 
Latency at iteration 64: 45 cycles 
value of r1 at iteration 72: 0.720000 
Latency at iteration 72: 53 cycles 
value of r1 at iteration 80: 0.800000 
Latency at iteration 80: 45 cycles 
value of r1 at iteration 88